In [1]:
from google.colab import files

print("📁 Por favor sube tu archivo kaggle.json")
print("👇 Click en 'Elegir archivos' y selecciona kaggle.json")
uploaded = files.upload()

📁 Por favor sube tu archivo kaggle.json
👇 Click en 'Elegir archivos' y selecciona kaggle.json


Saving kaggle (1).json to kaggle (1).json


In [4]:
!mkdir -p ~/.kaggle
!cp "kaggle (1).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions list

ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd       1029           False  
https://www.kaggle.com/competitions/jigsaw-agile-community-rules                 2025-10-24 06:59:00  Featured           100,000 Usd       2003           False  
https://www.kaggle.com/competitions/mitsui-commodity-prediction-challenge        2025-10-06 23:59:00  Featured           100,000 Usd       1624           False  
https://www.kaggle.com/competitions/google-code-golf-2025                        2025-10-30 23:59:00  Research           100,000 Usd        842           False  
https://www.kaggle.com/compe

In [7]:
# Descargar datos de la competencia
!kaggle competitions download -c llm-classification-finetuning
!unzip -q llm-classification-finetuning.zip

print("✅ Datos descargados y descomprimidos")

# Verificar archivos
import os
print("\n📁 Archivos disponibles:")
for f in os.listdir('.'):
    if f.endswith('.csv'):
        print(f"  - {f}")

  0% 0.00/57.0M [00:00<?, ?B/s]
100% 57.0M/57.0M [00:00<00:00, 1.59GB/s]
✅ Datos descargados y descomprimidos

📁 Archivos disponibles:
  - sample_submission.csv
  - train.csv
  - test.csv


In [8]:
# Verificar GPU
import torch
print(f"🔥 GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No hay GPU - El entrenamiento será MUY lento")

# Instalar librerías necesarias
!pip install -q transformers datasets accelerate scikit-learn textstat

print("\n✅ Librerías instaladas")

🔥 GPU disponible: True
GPU detectada: Tesla T4
Memoria GPU: 15.83 GB
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.1 MB/s eta 0:00:00

✅ Librerías instaladas


In [9]:
import pandas as pd
import numpy as np

# Cargar datos
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_df = pd.read_csv('sample_submission.csv')

print(f"📊 Train shape: {train_df.shape}")
print(f"📊 Test shape: {test_df.shape}")
print(f"\n✅ Primeras filas del train:")
print(train_df.head(2))

# Verificar distribución de clases
print(f"\n📈 Distribución de ganadores:")
print(f"  Model A gana: {train_df['winner_model_a'].sum()}")
print(f"  Model B gana: {train_df['winner_model_b'].sum()}")
print(f"  Empate: {train_df['winner_tie'].sum()}")

📊 Train shape: (57477, 9)
📊 Test shape: (3, 4)

✅ Primeras filas del train:
      id             model_a     model_b  \
0  30192  gpt-4-1106-preview  gpt-4-0613   
1  53567           koala-13b  gpt-4-0613   

                                              prompt  \
0  ["Is it morally right to try to have a certain...   
1  ["What is the difference between marriage lice...   

                                          response_a  \
0  ["The question of whether it is morally right ...   
1  ["A marriage license is a legal document that ...   

                                          response_b  winner_model_a  \
0  ["As an AI, I don't have personal beliefs or o...               1   
1  ["A marriage license and a marriage certificat...               0   

   winner_model_b  winner_tie  
0               0           0  
1               1           0  

📈 Distribución de ganadores:
  Model A gana: 20064
  Model B gana: 19652
  Empate: 17761


In [10]:
import json
import re

def extract_text(text_column):
    """Extraer texto de las columnas que están en formato de lista JSON"""
    if isinstance(text_column, str):
        try:
            # Convertir de string a lista
            text_list = json.loads(text_column)
            # Unir todos los elementos
            return ' '.join(text_list) if isinstance(text_list, list) else str(text_list)
        except:
            return str(text_column)
    return str(text_column)

# Aplicar a train
print("🔄 Procesando datos de entrenamiento...")
train_df['prompt_text'] = train_df['prompt'].apply(extract_text)
train_df['response_a_text'] = train_df['response_a'].apply(extract_text)
train_df['response_b_text'] = train_df['response_b'].apply(extract_text)

# Aplicar a test
print("🔄 Procesando datos de test...")
test_df['prompt_text'] = test_df['prompt'].apply(extract_text)
test_df['response_a_text'] = test_df['response_a'].apply(extract_text)
test_df['response_b_text'] = test_df['response_b'].apply(extract_text)

print("\n✅ Textos extraídos. Ejemplo:")
print(f"Prompt: {train_df['prompt_text'].iloc[0][:100]}...")
print(f"Response A: {train_df['response_a_text'].iloc[0][:100]}...")
print(f"Response B: {train_df['response_b_text'].iloc[0][:100]}...")

🔄 Procesando datos de entrenamiento...
🔄 Procesando datos de test...

✅ Textos extraídos. Ejemplo:
Prompt: Is it morally right to try to have a certain percentage of females on managerial positions? OK, does...
Response A: The question of whether it is morally right to aim for a certain percentage of females in managerial...
Response B: As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of ge...


In [11]:
import textstat

def calculate_features(df):
    """Calcular features de longitud y complejidad"""

    # Longitudes
    df['len_prompt'] = df['prompt_text'].str.len()
    df['len_response_a'] = df['response_a_text'].str.len()
    df['len_response_b'] = df['response_b_text'].str.len()

    # Diferencia de longitudes
    df['len_diff'] = df['len_response_a'] - df['len_response_b']
    df['len_ratio'] = df['len_response_a'] / (df['len_response_b'] + 1)

    # Complejidad léxica (Flesch Reading Ease - más alto = más fácil de leer)
    print("Calculando complejidad textual...")
    df['complexity_a'] = df['response_a_text'].apply(lambda x: textstat.flesch_reading_ease(x[:5000]))
    df['complexity_b'] = df['response_b_text'].apply(lambda x: textstat.flesch_reading_ease(x[:5000]))

    # Número de palabras
    df['words_a'] = df['response_a_text'].str.split().str.len()
    df['words_b'] = df['response_b_text'].str.split().str.len()

    return df

# Aplicar features
print("📊 Calculando features para train...")
train_df = calculate_features(train_df)

print("📊 Calculando features para test...")
test_df = calculate_features(test_df)

print("\n✅ Features creadas:")
feature_cols = ['len_prompt', 'len_response_a', 'len_response_b', 'len_diff',
                'len_ratio', 'complexity_a', 'complexity_b', 'words_a', 'words_b']
print(train_df[feature_cols].describe())

📊 Calculando features para train...
Calculando complejidad textual...
📊 Calculando features para test...
Calculando complejidad textual...

✅ Features creadas:
         len_prompt  len_response_a  len_response_b      len_diff  \
count  57477.000000    57477.000000    57477.000000  57477.000000   
mean     352.356612     1330.599422     1337.492058     -6.892635   
std     1025.378233     1462.378796     1484.782086   1060.492449   
min        3.000000        0.000000        0.000000 -51145.000000   
25%       48.000000      391.000000      396.000000   -413.000000   
50%       91.000000     1036.000000     1044.000000      0.000000   
75%      233.000000     1810.000000     1819.000000    405.000000   
max    32833.000000    53299.000000    52402.000000  42783.000000   

          len_ratio  complexity_a   complexity_b       words_a       words_b  
count  57477.000000  57477.000000   57477.000000  57477.000000  57477.000000  
mean       2.472271     46.774209      44.481684    212.3072

In [12]:
from sklearn.model_selection import train_test_split

# Crear texto combinado para el modelo
# Formato: [CLS] prompt [SEP] response_a [SEP] response_b [SEP]
train_df['combined_text'] = (
    train_df['prompt_text'] + ' [SEP] ' +
    train_df['response_a_text'] + ' [SEP] ' +
    train_df['response_b_text']
)

test_df['combined_text'] = (
    test_df['prompt_text'] + ' [SEP] ' +
    test_df['response_a_text'] + ' [SEP] ' +
    test_df['response_b_text']
)

# Crear labels (3 clases: model_a, model_b, tie)
# Convertir a una sola columna de clase
def get_winner_class(row):
    if row['winner_model_a'] == 1:
        return 0  # Model A gana
    elif row['winner_model_b'] == 1:
        return 1  # Model B gana
    else:
        return 2  # Empate

train_df['winner_class'] = train_df.apply(get_winner_class, axis=1)

# Split train/validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['combined_text'].values,
    train_df['winner_class'].values,
    test_size=0.1,
    random_state=42,
    stratify=train_df['winner_class'].values
)

print(f"✅ Train set: {len(train_texts)} ejemplos")
print(f"✅ Validation set: {len(val_texts)} ejemplos")
print(f"✅ Test set: {len(test_df)} ejemplos")
print(f"\n📊 Distribución de clases en train:")
print(pd.Series(train_labels).value_counts().sort_index())

✅ Train set: 51729 ejemplos
✅ Validation set: 5748 ejemplos
✅ Test set: 3 ejemplos

📊 Distribución de clases en train:
0    18057
1    17687
2    15985
Name: count, dtype: int64


In [13]:
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Cargar tokenizer y modelo
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # 3 clases: model_a, model_b, tie
)

# Mover modelo a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"✅ Modelo cargado en: {device}")
print(f"Parámetros del modelo: {model.num_parameters():,}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modelo cargado en: cuda
Parámetros del modelo: 66,955,779


In [14]:
# Tokenizar con límite de 512 tokens (límite de BERT)
def tokenize_function(texts):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

print("🔄 Tokenizando datos de entrenamiento...")
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

print("🔄 Tokenizando datos de test...")
test_encodings = tokenizer(
    test_df['combined_text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

# Crear datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

print(f"✅ Tokenización completa")
print(f"   Train dataset: {len(train_dataset)} ejemplos")
print(f"   Val dataset: {len(val_dataset)} ejemplos")

🔄 Tokenizando datos de entrenamiento...
🔄 Tokenizando datos de test...
✅ Tokenización completa
   Train dataset: 51729 ejemplos
   Val dataset: 5748 ejemplos


In [17]:
# Desactivar wandb completamente
import os
os.environ["WANDB_DISABLED"] = "true"

print("✅ Wandb desactivado")

✅ Wandb desactivado


In [18]:
# Función para calcular métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# Configuración de entrenamiento (parámetro corregido)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="steps",  # CAMBIO: era evaluation_strategy
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    fp16=True,
)

# Crear trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("Iniciando entrenamiento...")
print("Esto tomará aproximadamente 10-15 minutos")
print("-" * 50)

# Entrenar
trainer.train()

print("\nEntrenamiento completado!")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Iniciando entrenamiento...
Esto tomará aproximadamente 10-15 minutos
--------------------------------------------------


Step,Training Loss,Validation Loss,Accuracy,F1
500,1.101800,1.097074,0.350209,0.182971
1000,1.089200,1.089872,0.374565,0.286976
1500,1.088100,1.092536,0.373869,0.291685
2000,1.090200,1.085015,0.382046,0.320281
2500,1.093800,1.089080,0.361517,0.285498
3000,1.087300,1.088828,0.382742,0.284190
3500,1.076900,1.093998,0.385351,0.351230
4000,1.071400,1.094904,0.379784,0.331931
4500,1.067300,1.089346,0.383090,0.291851
5000,1.074600,1.088465,0.381350,0.299337



Entrenamiento completado!


In [19]:
# Evaluar en validation set
print("📊 Evaluando modelo en validation set...")
eval_results = trainer.evaluate()
print(f"Resultados finales:")
print(f"  Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"  F1 Score: {eval_results['eval_f1']:.4f}")

# Hacer predicciones en test
print("\n🔮 Generando predicciones para test set...")
test_dataset_pred = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
})

predictions = trainer.predict(test_dataset_pred)
probs = torch.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

print(f"✅ Predicciones generadas")
print(f"Shape de probabilidades: {probs.shape}")
print(f"\nPrimera predicción (probabilidades):")
print(f"  Model A: {probs[0][0]:.4f}")
print(f"  Model B: {probs[0][1]:.4f}")
print(f"  Tie: {probs[0][2]:.4f}")

📊 Evaluando modelo en validation set...


Resultados finales:
  Accuracy: 0.3820
  F1 Score: 0.3203

🔮 Generando predicciones para test set...
✅ Predicciones generadas
Shape de probabilidades: (3, 3)

Primera predicción (probabilidades):
  Model A: 0.3142
  Model B: 0.2757
  Tie: 0.4101


In [20]:
# Crear submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': probs[:, 0],
    'winner_model_b': probs[:, 1],
    'winner_tie': probs[:, 2]
})

# Guardar
submission.to_csv('submission.csv', index=False)

print("✅ Submission creado!")
print("\n📄 Primeras filas del submission:")
print(submission)

# Verificar que las probabilidades suman 1
print(f"\n🔍 Verificación (suma de probabilidades):")
print(submission[['winner_model_a', 'winner_model_b', 'winner_tie']].sum(axis=1))

✅ Submission creado!

📄 Primeras filas del submission:
        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.314190        0.275703    0.410107
1   211333        0.368587        0.353883    0.277529
2  1233961        0.371351        0.367339    0.261310

🔍 Verificación (suma de probabilidades):
0    1.0
1    1.0
2    1.0
dtype: float32


In [25]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>